In [13]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
import math
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
import seaborn as sns

In [2]:
df = pd.read_csv('./merged data.csv')

In [3]:
def format_df(df): # formats df by getting rid of extra columns and making negative delays = 0
  df_copy = df.copy()
  for i in range(len(df_copy['DEP_DELAY'])):
    if df_copy['DEP_DELAY'][i] < 0:    
      df_copy['DEP_DELAY'][i]=0
  df_copy.drop(['Unnamed: 0','timestamp_utc'],axis=1)
  return df_copy

In [4]:
formatted_df = format_df(df)

/var/folders/dk/ntqqxtcj77d6kpjl7pz6dbbr0000gn/T/ipykernel_95540/3960108429.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['DEP_DELAY'][i]=0


In [15]:
sns.pairplot(data=formatted_df)
plt.savefig("./pairplotforweather.png")

In [5]:
def predict_mlp_classifier_precip_rate(df):
  df_train = df.iloc[:5629, :] #x is precipitation and y is delay
  df_test = df.iloc[5629:, :]
  for i in range(len(df['DEP_DELAY'])): # for this model, if the flight was cancelled, we set the delay to -1
    if math.isnan(df['DEP_DELAY'][i]):
      df['DEP_DELAY'][i]=-1
  df_train_y = df_train[['DEP_DELAY']]
  df_train_x = df_train[['precip_rate']]
  model = MLPClassifier(hidden_layer_sizes=(8,9,9), activation='relu', max_iter=5000)
  model.fit(df_train_x, df_train_y)
  print("score:", model.score(df_train_x, df_train_y)) # this is for comparing delay to precipitation (where delay =0 means no delay and delay = -1 means cancelled)

In [6]:
predict_mlp_classifier_precip_rate(formatted_df) #score: 0.46704565642209983

/var/folders/dk/ntqqxtcj77d6kpjl7pz6dbbr0000gn/T/ipykernel_95540/2702389595.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DEP_DELAY'][i]=-1
/opt/homebrew/Caskroom/miniforge/base/envs/test_env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


score: 0.46704565642209983


In [7]:
def predict_mlp_classifier_temp(df):
  df_train = df.iloc[:5629, :] #x is temp and y is delay
  df_test = df.iloc[5629:, :]
  for i in range(len(df['DEP_DELAY'])):
    if df['DEP_DELAY'][i]==-1:
      df['DEP_DELAY'][i]=300
  df_train_y = df_train[['DEP_DELAY']]
  df_train_x = df_train[['temp']]
  model = MLPClassifier(hidden_layer_sizes=(8,9,9), activation='relu', max_iter=5000)
  model.fit(df_train_x, df_train_y)
  print("score:", model.score(df_train_x, df_train_y)) # this is for comparing delay to temp 

In [8]:
predict_mlp_classifier_temp(formatted_df) #score: 0.46722330786995914

/var/folders/dk/ntqqxtcj77d6kpjl7pz6dbbr0000gn/T/ipykernel_95540/2374956392.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DEP_DELAY'][i]=300
/opt/homebrew/Caskroom/miniforge/base/envs/test_env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


score: 0.46704565642209983


In [9]:
def predict_logistic_regression(df):
  features = ['precip_rate', 'wind_spd']
  X = df[features]
  df['cancelled'] = ''
  for i in range(len(df['DEP_DELAY'])): # adding 'cancelled' column
    if(np.isnan(df['DEP_DELAY'][i])):
      df['cancelled'][i]=True
      df['DEP_DELAY'][i] = -1
    #else:
      #df['cancelled'][i]=False
  #y = df['cancelled']
  y = df['DEP_DELAY']
  print(y)
  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)
  log = LogisticRegression(class_weight='balanced')
  log.fit(X_scaled,y)
  print("score:", log.score(X_scaled,y))
  return df #score: 0.0014212115828744004

In [10]:
predict_logistic_regression(df)

0        -5.0
1        36.0
2        47.0
3        28.0
4         7.0
         ... 
11253    -7.0
11254    -8.0
11255    -4.0
11256   -10.0
11257    -6.0
Name: DEP_DELAY, Length: 11258, dtype: float64


/var/folders/dk/ntqqxtcj77d6kpjl7pz6dbbr0000gn/T/ipykernel_95540/2109250230.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cancelled'][i]=True
/var/folders/dk/ntqqxtcj77d6kpjl7pz6dbbr0000gn/T/ipykernel_95540/2109250230.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DEP_DELAY'][i] = -1


score: 0.0012435601350151003


/opt/homebrew/Caskroom/miniforge/base/envs/test_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Unnamed: 0,timestamp_local,DEP_DELAY,app_temp,azimuth,clouds,dewpt,dhi,dni,elev_angle,...,ts,uv,vis,weather/description,weather/code,weather/icon,wind_dir,wind_gust_spd,wind_spd,cancelled
0,0,2018-01-01T15:15:00,-5.0,-14.9,210.66,0,-22.5,76.57,722.62,20.00,...,1514798100,1.8,16,Clear Sky,800,c01d,125,0.8,0.8,
1,1,2018-01-01T15:15:00,36.0,-14.9,210.66,0,-22.5,76.57,722.62,20.00,...,1514798100,1.8,16,Clear Sky,800,c01d,125,0.8,0.8,
2,2,2018-01-01T15:15:00,47.0,-14.9,210.66,0,-22.5,76.57,722.62,20.00,...,1514798100,1.8,16,Clear Sky,800,c01d,125,0.8,0.8,
3,3,2018-01-01T15:15:00,28.0,-14.9,210.66,0,-22.5,76.57,722.62,20.00,...,1514798100,1.8,16,Clear Sky,800,c01d,125,0.8,0.8,
4,4,2018-01-01T15:15:00,7.0,-14.9,210.66,0,-22.5,76.57,722.62,20.00,...,1514798100,1.8,16,Clear Sky,800,c01d,125,0.8,0.8,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11253,11253,2018-01-30T20:45:00,-7.0,-24.6,269.70,4,-22.5,0.00,0.00,-27.59,...,1517323500,0.0,16,Few clouds,801,c02n,102,1.7,1.6,
11254,11254,2018-01-30T20:45:00,-8.0,-24.6,269.70,4,-22.5,0.00,0.00,-27.59,...,1517323500,0.0,16,Few clouds,801,c02n,102,1.7,1.6,
11255,11255,2018-01-30T11:15:00,-4.0,-19.8,147.41,100,-18.2,84.88,782.17,24.88,...,1517289300,0.7,16,Light snow,600,s01d,132,1.7,1.6,
11256,11256,2018-01-30T05:00:00,-10.0,-21.7,81.52,98,-19.1,0.00,0.00,-37.73,...,1517266800,0.0,16,Overcast clouds,804,c04n,118,2.5,2.4,


In [11]:
def predict_logistic_regression_with_negative_delays_as_0(df):
  features = ['precip_rate', 'wind_spd']
  X = df[features]
  df['cancelled'] = ''
  for i in range(len(df['DEP_DELAY'])): # adding 'cancelled' column
    if(np.isnan(df['DEP_DELAY'][i])):
      df['cancelled'][i]=True
      df['DEP_DELAY'][i] = -1
    if(df["DEP_DELAY"][i] < 0):
      df["DEP_DELAY"][i] = 0
    #else:
      #df['cancelled'][i]=False
  #y = df['cancelled']
  y = df['DEP_DELAY']
  print(y)
  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)
  log = LogisticRegression(class_weight='balanced', max_iter=10000)
  log.fit(X_scaled,y)
  print("score:", log.score(X_scaled,y))
  return df #score: 0.0013323858589447503

In [12]:
predict_logistic_regression_with_negative_delays_as_0(formatted_df)

0         0.0
1        36.0
2        47.0
3        28.0
4         7.0
         ... 
11253     0.0
11254     0.0
11255     0.0
11256     0.0
11257     0.0
Name: DEP_DELAY, Length: 11258, dtype: float64
score: 0.0015988630307337006


,Unnamed: 0,timestamp_local,DEP_DELAY,app_temp,azimuth,clouds,dewpt,dhi,dni,elev_angle,...,ts,uv,vis,weather/description,weather/code,weather/icon,wind_dir,wind_gust_spd,wind_spd,cancelled
0,0,2018-01-01T15:15:00,0.0,-14.9,210.66,0,-22.5,76.57,722.62,20.00,...,1514798100,1.8,16,Clear Sky,800,c01d,125,0.8,0.8,
1,1,2018-01-01T15:15:00,36.0,-14.9,210.66,0,-22.5,76.57,722.62,20.00,...,1514798100,1.8,16,Clear Sky,800,c01d,125,0.8,0.8,
2,2,2018-01-01T15:15:00,47.0,-14.9,210.66,0,-22.5,76.57,722.62,20.00,...,1514798100,1.8,16,Clear Sky,800,c01d,125,0.8,0.8,
3,3,2018-01-01T15:15:00,28.0,-14.9,210.66,0,-22.5,76.57,722.62,20.00,...,1514798100,1.8,16,Clear Sky,800,c01d,125,0.8,0.8,
4,4,2018-01-01T15:15:00,7.0,-14.9,210.66,0,-22.5,76.57,722.62,20.00,...,1514798100,1.8,16,Clear Sky,800,c01d,125,0.8,0.8,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11253,11253,2018-01-30T20:45:00,0.0,-24.6,269.70,4,-22.5,0.00,0.00,-27.59,...,1517323500,0.0,16,Few clouds,801,c02n,102,1.7,1.6,
11254,11254,2018-01-30T20:45:00,0.0,-24.6,269.70,4,-22.5,0.00,0.00,-27.59,...,1517323500,0.0,16,Few clouds,801,c02n,102,1.7,1.6,
11255,11255,2018-01-30T11:15:00,0.0,-19.8,147.41,100,-18.2,84.88,782.17,24.88,...,1517289300,0.7,16,Light snow,600,s01d,132,1.7,1.6,
11256,11256,2018-01-30T05:00:00,0.0,-21.7,81.52,98,-19.1,0.00,0.00,-37.73,...,1517266800,0.0,16,Overcast clouds,804,c04n,118,2.5,2.4,
